In [1]:
# External libraries
import numpy as np
import os

# Drake dependencies
from pydrake.all import (
    DiagramBuilder,
    Simulator,
    AddDefaultVisualization,
    StartMeshcat,
    MultibodyPlant,
    Demultiplexer,
    ConstantVectorSource,
    Parser,
    AddMultibodyPlantSceneGraph,
    ConstantVectorSource,
    DiagramBuilder,
    JointSliders,
    MeshcatVisualizer,
    MultibodyPlant,
    MultibodyPositionToGeometryPose,
    Multiplexer,
    Parser,
    PrismaticJoint,
    SceneGraph,
    Simulator,
    SpatialInertia,
    Sphere,
    StartMeshcat,
    UnitInertia,
    MeshcatVisualizerParams,
    LoadModelDirectives,
    LoadModelDirectivesFromString,
    ProcessModelDirectives,
    DiscreteContactApproximation
)

import manipulation
from manipulation.meshcat_utils import MeshcatSliders, StopButton
from manipulation.utils import RenderDiagram, ConfigureParser

meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7002


In [ ]:
model_directive = f"""
directives:
- add_model:
    name: food
    file: file://{os.getcwd()}/Models/cube_food.sdf
- add_weld:
    parent: world
    child: food::cube_food
- add_model:
    name: skewer
    file: file://{os.getcwd()}/Models/skewer_5mm.sdf
    default_free_body_pose:
        skewer_5mm:""" + """
            translation: [-0.052, -0.052, 0.1]
            rotation: !Rpy { deg: [0, 0, 90]}
"""

In [3]:
meshcat.DeleteAddedControls()

builder = DiagramBuilder()

plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=2e-4)
plant.set_discrete_contact_approximation(DiscreteContactApproximation.kLagged)
directives = LoadModelDirectivesFromString(model_directive)
parser = Parser(plant)
parser.package_map().Add("manipulation", manipulation.__path__[0] + "/models/")
models = ProcessModelDirectives(directives, plant, parser)
plant.Finalize()

AddDefaultVisualization(builder=builder, meshcat=meshcat)

In [4]:
diagram = builder.Build()
simulator = Simulator(diagram)

In [5]:
meshcat.StartRecording()
simulator.AdvanceTo(2.0)
meshcat.PublishRecording()
print(simulator.get_actual_realtime_rate())

0.06209830016537631
